In [1]:
import requests
import pandas as pd
import datetime as dt
import time
import praw
import numpy as np

In [2]:
base_url = 'https://api.pushshift.io/reddit/submission/search'

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
current_time = round(time.time())

In [5]:
params = {
    'subreddit' : 'superhoops',
    'limit' : 1000,
    'until' : current_time
    
}

In [6]:
res = requests.get(base_url, params)

In [7]:
df = pd.DataFrame(res.json()['data'])
df.shape

(1000, 101)

In [8]:
# def get_submission_ids(subreddit):
#     submissions = []
#     url = f"https://api.pushshift.io/reddit/search/submission?subreddit={subreddit}&size=200"
#     start = dt.datetime(2011, 1, 1).timestamp()  
#     num_posts = 0
#     all_posts = []
#     while True:
#         start_time = time.time()
#         res = requests.get(url)
#         print(res.status_code)
#         try:
#             posts = pd.DataFrame(res.json()['data'])
#             all_posts.append(posts)
#             num_posts += len(posts)
#             end_time = time.time()
#             time_to = end_time - start_time
#             total_time += time_to
#         except:
#             pass
#         print(f"time to scrape: {time_to}; num posts: {num_posts}")
#     df = pd.DataFrame(all_posts)
#     return df[['id']]


In [9]:
def pull_reddit_posts(subreddit):
    """
    This function is designed to take in a set of subreddits, connect to the pushshift api,
    and pull all submissions by setting a lowerbound of choice as start_utc. Utilizes a while
    loop to get posts using requests and has checks inside for:
    1. If data is not in the response json
    2. If the last post of the most recent pull is less than the lower bounds
    Once all posts have been collected and/or the loop breaks, save all posts as a dataframe
    and save to csv in project data folder.

    """
    posts = []

    
    print("this is subreddit", subreddit)
    # save URL based on current subreddit
    url = f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=500"
    # set date as lower bounds, see if we can limit calls by only pulling since 2015
    start_utc = dt.datetime(2011, 1, 1).timestamp()
    print("start_utc", start_utc)
    while True:
        start_time = time.time()
        # call requests, save response
        response = requests.get(url)
        # save json as data
        data = response.json()
        # if data key not in response, break loop
        if len(data['data']) == 0:
            break
        # else, add data to posts list
        posts += data['data']
        print("posts length", len(posts))
        # pull last post in most recent data pull
        last_post = data['data'][-1]
        # save created_utc for gate and next url
        created_utc = last_post['created_utc']
        print("created_utc", created_utc)
        # current last post utc less than lower bounds, break loop
        if created_utc < start_utc:
            break
        # if loop hasn't broken, set url for next pull passing created_utc as new 'before' parameter
        url = f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=500&before={created_utc}"
        end_time = time.time()
        print("total time taken this loop: ", end_time-start_time)

    df = pd.DataFrame(posts)
    return df

In [10]:
df_n = pull_reddit_posts('superhoops')

this is subreddit superhoops
start_utc 1293840000.0
posts length 500
created_utc 1612184284
total time taken this loop:  2.853407144546509
posts length 1000
created_utc 1576006248
total time taken this loop:  2.9799087047576904
posts length 1500
created_utc 1535979215
total time taken this loop:  2.9519600868225098
posts length 2000
created_utc 1502147101
total time taken this loop:  1.857154130935669
posts length 2500
created_utc 1475073051
total time taken this loop:  2.7514848709106445
posts length 3000
created_utc 1455535881
total time taken this loop:  2.422658681869507
posts length 3500
created_utc 1440764885
total time taken this loop:  3.651289939880371
posts length 4000
created_utc 1431277246
total time taken this loop:  1.687751293182373
posts length 4500
created_utc 1418985528
total time taken this loop:  2.2887020111083984
posts length 5000
created_utc 1408890147
total time taken this loop:  1.8755199909210205
posts length 5500
created_utc 1398877354
total time taken this l

In [11]:
df_n

,subreddit,selftext,author_fullname,gilded,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,thumbnail_height,top_awarded_type,hide_score,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,total_awards_received,media_embed,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,score,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,link_flair_type,wls,removed_by_category,author_flair_type,domain,allow_live_comments,suggested_sort,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,removed_by,distinguished,subreddit_id,link_flair_background_color,id,is_robot_indexable,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,retrieved_utc,updated_utc,utc_datetime_str,post_hint,preview,link_flair_template_id,crosspost_parent_list,crosspost_parent,is_gallery,media_metadata,gallery_data,poll_data,author_cakeday,author_created_utc,retrieved_on,edited_on,removal_reason,approved_by,created,downs,likes,mod_reports,num_reports,post_categories,report_reasons,rte_mode,saved,selftext_html,ups,user_reports,brand_safe,approved_at_utc,banned_at_utc,from,from_id,from_kind
0,superhoops,,t2_frdzs,0.0,QPR themed wedding,[],r/superhoops,False,6.0,None,NaN,NaN,True,False,dark,1.0,transparent,public,0.0,{},NaN,60b296e6-2cdd-11eb-9077-0eaf553ab7a7,False,None,False,False,NaN,{},None,1,False,False,default,False,None,[],{},NaN,False,text,6.0,None,text,twitter.com,False,NaN,https://twitter.com/flapjacq88/status/16403096...,NaN,False,True,True,False,False,[],[],False,True,False,False,England R's 🏴󠁧󠁢󠁥󠁮󠁧󠁿,[],NaN,None,t5_2swvo,,123pxf7,True,GaxZE,NaN,0,True,all_ads,False,False,dark,/r/superhoops/comments/123pxf7/qpr_themed_wedd...,all_ads,False,https://twitter.com/flapjacq88/status/16403096...,2575.0,1679930111,0.0,None,False,1679930125,1679930125,2023-03-27 15:15:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,superhoops,Hey guys. I’m making a Queens Park Rangers Car...,t2_cu65p6m1,0.0,QPR Career mode,[],r/superhoops,False,6.0,None,NaN,NaN,True,False,dark,1.0,None,public,0.0,{},NaN,None,False,None,False,False,NaN,{},None,1,False,False,self,False,None,[],{},NaN,True,text,6.0,None,text,self.superhoops,False,NaN,NaN,NaN,False,True,True,False,False,[],[],False,True,False,False,None,[],NaN,None,t5_2swvo,,121vebo,True,Historical_Giraffe45,NaN,0,True,all_ads,False,False,None,/r/superhoops/comments/121vebo/qpr_career_mode/,all_ads,False,https://www.reddit.com/r/superhoops/comments/1...,2576.0,1679770444,0.0,None,False,1679770461,1679770462,2023-03-25 18:54:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,superhoops,,t2_4je6l,0.0,Queens Park Rangers C Club hospitality - REVIE...,[],r/superhoops,False,6.0,,105.0,NaN,True,False,dark,1.0,transparent,public,0.0,"{'content': '&lt;iframe width=""267"" height=""20...",140.0,20dc4b9a-2cdb-11eb-89d3-0e959dd058df,False,"{'type': 'youtube.com', 'oembed': {'provider_u...",False,False,NaN,"{'content': '&lt;iframe width=""267"" height=""20...",Media,1,False,False,https://a.thumbs.redditmedia.com/QUFGxj8rD92me...,False,None,[],{},NaN,False,text,6.0,None,text,youtube.com,False,NaN,https://www.youtube.com/watch?v=tamSJg2Ezfw,NaN,False,True,True,False,False,[],[],False,True,False,False,Canadian R's 🇨🇦,[],NaN,None,t5_2swvo,,11zoijt,True,peetss,NaN,0,True,all_ads,False,False,dark,/r/superhoops/comments/11zoijt/queens_

In [12]:
df_n.selftext.replace('', np.nan, inplace = True)

In [13]:
df_n.selftext.fillna(df_n.title, inplace = True)

In [14]:
df_n

,subreddit,selftext,author_fullname,gilded,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,thumbnail_height,top_awarded_type,hide_score,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,total_awards_received,media_embed,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,score,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,link_flair_type,wls,removed_by_category,author_flair_type,domain,allow_live_comments,suggested_sort,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,removed_by,distinguished,subreddit_id,link_flair_background_color,id,is_robot_indexable,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,retrieved_utc,updated_utc,utc_datetime_str,post_hint,preview,link_flair_template_id,crosspost_parent_list,crosspost_parent,is_gallery,media_metadata,gallery_data,poll_data,author_cakeday,author_created_utc,retrieved_on,edited_on,removal_reason,approved_by,created,downs,likes,mod_reports,num_reports,post_categories,report_reasons,rte_mode,saved,selftext_html,ups,user_reports,brand_safe,approved_at_utc,banned_at_utc,from,from_id,from_kind
0,superhoops,QPR themed wedding,t2_frdzs,0.0,QPR themed wedding,[],r/superhoops,False,6.0,None,NaN,NaN,True,False,dark,1.0,transparent,public,0.0,{},NaN,60b296e6-2cdd-11eb-9077-0eaf553ab7a7,False,None,False,False,NaN,{},None,1,False,False,default,False,None,[],{},NaN,False,text,6.0,None,text,twitter.com,False,NaN,https://twitter.com/flapjacq88/status/16403096...,NaN,False,True,True,False,False,[],[],False,True,False,False,England R's 🏴󠁧󠁢󠁥󠁮󠁧󠁿,[],NaN,None,t5_2swvo,,123pxf7,True,GaxZE,NaN,0,True,all_ads,False,False,dark,/r/superhoops/comments/123pxf7/qpr_themed_wedd...,all_ads,False,https://twitter.com/flapjacq88/status/16403096...,2575.0,1679930111,0.0,None,False,1679930125,1679930125,2023-03-27 15:15:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,superhoops,Hey guys. I’m making a Queens Park Rangers Car...,t2_cu65p6m1,0.0,QPR Career mode,[],r/superhoops,False,6.0,None,NaN,NaN,True,False,dark,1.0,None,public,0.0,{},NaN,None,False,None,False,False,NaN,{},None,1,False,False,self,False,None,[],{},NaN,True,text,6.0,None,text,self.superhoops,False,NaN,NaN,NaN,False,True,True,False,False,[],[],False,True,False,False,None,[],NaN,None,t5_2swvo,,121vebo,True,Historical_Giraffe45,NaN,0,True,all_ads,False,False,None,/r/superhoops/comments/121vebo/qpr_career_mode/,all_ads,False,https://www.reddit.com/r/superhoops/comments/1...,2576.0,1679770444,0.0,None,False,1679770461,1679770462,2023-03-25 18:54:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,superhoops,Queens Park Rangers C Club hospitality - REVIE...,t2_4je6l,0.0,Queens Park Rangers C Club hospitality - REVIE...,[],r/superhoops,False,6.0,,105.0,NaN,True,False,dark,1.0,transparent,public,0.0,"{'content': '&lt;iframe width=""267"" height=""20...",140.0,20dc4b9a-2cdb-11eb-89d3-0e959dd058df,False,"{'type': 'youtube.com', 'oembed': {'provider_u...",False,False,NaN,"{'content': '&lt;iframe width=""267"" height=""20...",Media,1,False,False,https://a.thumbs.redditmedia.com/QUFGxj8rD92me...,False,None,[],{},NaN,False,text,6.0,None,text,youtube.com,False,NaN,https://www.youtube.com/watch?v=tamSJg2Ezfw,NaN,False,True,True,False,False,[],[],False,True,False,False,Canadian R's 🇨🇦,[],NaN,None,t5_2swvo,,11zoijt,True,peetss,NaN,0,T

In [15]:
df_n['length'] = df_n.selftext.str.split().apply(lambda x: len(x))
long = df_n['length'] > 10
df_n = df_n[long]

In [16]:
df_n.length.min()

11

In [17]:
df_n.shape

(3142, 121)

In [18]:
df_n.selftext.value_counts(ascending = False)

QPR ace sparks war with West Ham striker: "Something doesn't seem right!"                                         2
Clint Hill urges Harry Redknapp not to sign too many players in Jan transfer window.                              2
Exclusive - Neil Warnock not in line for permanent Queens Park Rangers job, he tells talkSPORT                    2
QPR trainer Kevin Hitchcock embraces his son Tom after he scores the winner for QPR.                              2
MAN OF THE MATCH: YUN SUK-YOUNG (경기의 사람 : 석 영 윤)                                                                  2
                                                                                                                 ..
Leroy Fer is on the verge of a loan move to Swansea until the end of the season.                                  1
QPR have completed the signing of wide-man Abdenasser El Khayati from Burton Albion                               1
Young QPR striker Ben Pattie has extended his loan spell with Hendon    

In [34]:
to_keep = df_n[[
    'subreddit',
    'selftext',
    'score',
    'length',
    'num_comments',
    'utc_datetime_str',
    'id'
]]

In [35]:
df_posts = to_keep

In [36]:
df_posts.utc_datetime_str = df_posts.utc_datetime_str.str[:10]

/var/folders/5h/p8nqld_13hg44lctlhg2ns8c0000gn/T/ipykernel_4473/3181375314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_posts.utc_datetime_str = df_posts.utc_datetime_str.str[:10]


In [37]:
df_posts

,subreddit,selftext,score,length,num_comments,utc_datetime_str,id
1,superhoops,Hey guys. I’m making a Queens Park Rangers Car...,1,35,0,2023-03-25,121vebo
4,superhoops,Gareth Ainsworth: EFL financial woes not helpe...,1,12,0,2023-03-22,11yaovc
11,superhoops,Watford lose Wilder's first game as Rangers en...,1,11,0,2023-03-12,11p6t24
12,superhoops,Wesley Hoedt refuses to shake hands with Garet...,1,12,0,2023-03-11,11opn12
15,superhoops,Tyler Roberts returning to Leeds to review inj...,1,17,0,2023-03-10,11nchpj
...,...,...,...,...,...,...,...
8189,superhoops,Wonder what he has on QPR for the 23rd.\n\nFro...,1,82,0,2011-09-28,kufkn
8192,superhoops,BBC - Football Tactics: Goal-shy QPR in need o...,1,11,0,2011-09-26,ks88o
8195,superhoops,"26 Sept 2011: 'If you are hurting inside, you ...",1,37,0,2011-09-26,ks2t3
8196,superhoops,"Open All R's: A fairly new, independent weekly...",3,28,1,2011-09-26,krxxx


In [38]:
df_posts.num_comments.mean()

3.0642902609802674

In [39]:
df_posts.to_csv('qpr_posts.csv', index = False)